In [1]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
data = pd.read_csv("norm_data__non_log.txt",sep='\t')
#data

In [ ]:
# Extract labels

def extract_label(list): 
    number = '[0-9]'
    symbol = '_'
    head = 'Sample'
    list = [re.sub(number, '', i) for i in list] 
    list = [re.sub(symbol, '', i) for i in list] 
    list = [re.sub(head, '', i) for i in list] 
    return list

labels = list(data)
labels = extract_label(labels)
labels = np.ravel(labels)

data=data.T

mapping = {'Non-LCa':-1,'LCa':1}
data["targets"] = labels
data["targets"] = data["targets"].map(mapping)
#data

#X = data.iloc[:,0:1183].as_matrix()
#y = data["targets"].as_matrix()
#print(X.shape)
#print(y.shape)


In [ ]:
# Scale Features in [0,1]

scaler = MinMaxScaler() 

X = data.iloc[:,0:1183]
data_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

#scale targets too
#data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns) 

data_scaled
X = data_scaled.as_matrix()
y = data["targets"].as_matrix
# type(X)
# X.shape

In [ ]:
# X = data.iloc[:,0:1183].as_matrix()
# y = data["targets"].as_matrix()
# #print(X.shape)
# #print(y.shape)

In [ ]:
# pdf with KDE

# from sklearn.neighbors import KernelDensity
# kde = KernelDensity(kernel='gaussian', bandwidth=0.25).fit(X)
# kde.sample(3)

In [ ]:
# score_samples returns the log of the probability density

# print(X[11])
# print(X[200])

# print(kde.score_samples(X[11].reshape(1,-1)),
#       kde.score_samples(X[180].reshape(1,-1)),
#       kde.score_samples(X[18].reshape(1,-1)),
#       kde.score_samples(X[10].reshape(1,-1)),
#       kde.score_samples(X[100].reshape(1,-1)),
#       kde.score_samples(X[1000].reshape(1,-1)),)

# logprob=kde.score_samples(X)
# print(logprob)
# print(np.exp(logprob))

In [ ]:
# Try one pair

# i = 1000
# j = 100

# from sklearn.neighbors import KernelDensity

# xgrid = np.linspace(0, 1, 1000)
# kde_i = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[i]].reshape(-1,1))
# kde_j = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[j]].reshape(-1,1))
# sample_prob_i = kde_i.score_samples(xgrid.reshape(-1,1))
# sample_prob_j = kde_j.score_samples(xgrid.reshape(-1,1))

# #diff = np.sum((np.exp(sample_prob_i)-np.exp(sample_prob_j))**2)
# diff = np.sum(((sample_prob_i)-(sample_prob_j))**2)
# print(diff)


# print(np.exp(sample_prob_i))
# print(np.exp(sample_prob_j))
# print(sample_prob_i)
# print(sample_prob_j)

In [ ]:
# All pairs

from sklearn.neighbors import KernelDensity

xgrid = np.linspace(0, 1, 10)
#sim_table = pd.DataFrame()
num_SimilarFeature = []

for i in range(0,1183):
    diff_i = []
    kde_i = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[i]].reshape(-1,1))
    sample_prob_i = kde_i.score_samples(xgrid.reshape(-1,1))
    
    for j in range(0,1183):
        #if j!=i: 

        kde_j = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[j]].reshape(-1,1))
        sample_prob_j = kde_j.score_samples(xgrid.reshape(-1,1))
        diff = np.sum(((sample_prob_i)-(sample_prob_j))**2)   
#         print(i,j,diff)
        diff_i.append(diff)
#     print(diff_i)
#     the SimilarFeature includes itself
    
    num_SimilarFeature_i = np.sum(1 for s in diff_i if s<=1)
    num_SimilarFeature.append(num_SimilarFeature_i)
print(num_SimilarFeature)
#     print(num_SimilarFeature_i)    
#     sim_table[i] = diff_i
# sim_table         

In [ ]:
with open('sim_density.txt', 'w') as f:
    for item in num_SimilarFeature:
        f.write("%s\n" % item)

f.close()

In [ ]:
# Extract top-k distinct features 

k1 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 200]
# print('best',len(k),':\n',k)

k2 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 230]
# print('best',len(k),':\n',k)

k3 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 250]
# print('best',len(k),':\n',k)

with open('sim_density_top22.txt', 'w') as f:
    for item in k1:
        f.write("%s\n" % item)
f.close()
        
with open('sim_density_top30.txt', 'w') as f:
    for item in k2:
        f.write("%s\n" % item)
f.close()
        
with open('sim_density_top47.txt', 'w') as f:
    for item in k3:
        f.write("%s\n" % item)
f.close()

In [ ]:
# Reduce Dimension

# k2 = []
# with open("sim_density_top30.txt") as f:
#     for line in f:
#         k2.append(line)
        
# k2 = [float(i) for i in k2]        
        
# X_new = data.iloc[:,k2]
# print(X_new.shape)